# Exploratory Data Analysis

### Synopsis
In this assignment, we’re providing you with data from an experiment our product team ran in 2020. The team wanted to test the impact of loading item pages in a new tab in the user’s browser. A new tab would be triggered any time a user clicked on an item on the site (on the homepage, in search, on comparison shopping pages, etc.).

For most experiments at Reverb, we look at a two step conversion funnel, the first step being the event in which the user sees the feature change, and the second being a purchase, and we measure conversion through the funnel within a two day window. Revenue is generally our primary success metric, and we revenue lift by measuring the impact on both conversion rate and average value of those conversions. We generally apply a significance level of 0.1.

### Data
Attached are two datasets. The first is a dataset of events fired when a user clicked on a listing card. When experiment group = 0, the item page was opened in the same tab, and when experiment group = 1, the item page was opened in a new tab. Each event has an attached cookie id and a reverb user id. These user ids are attached to the data set retroactively, meaning that if we’ve ever associated a user id with a cookie, we’ll add it to the event (even if we tied them together after the event took place). The second dataset is all purchases during the time period.

### Task
The task is to analyze this experiment using the data provided.

Explain your methodology for analyzing the data.

Analyze the experiment
* Q1: How did conversion rate change?
* Q1A: How confident can we be in this answer?   
* Q2: How did conversion value change? 
* Q2A: How confident can we be in this answer?
* Q3: Are there any differences in behavior between logged in and logged out users?

What is your recommendation for the product team? What’s the estimated impact on revenue?

In [108]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
from tqdm.auto import tqdm
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

import matplotlib.pyplot as plt

In [3]:
ls -l ./Reverb_Sr\ Data\ Scientist_Data_Exercise

total 192264
-rw-r--r--@ 1 cambo  staff  14280428 Sep 11 14:41 conversion_data.csv
-rw-r--r--@ 1 cambo  staff  84154152 Sep 11 14:41 event_data.csv


In [4]:
data_dir = './Reverb_Sr Data Scientist_Data_Exercise/'

conversion_df = pd.read_csv('./{}/conversion_data.csv'.format(data_dir))
event_df = pd.read_csv('./{}/event_data.csv'.format(data_dir))

In [5]:
conversion_df.head()

,created_at,user_id,order_id,listing_id,revenue
0,2020-08-24 19:05:34.550,1167890,1821377,3967005,4.99
1,2020-08-14 12:21:24.985,4435269,9241629,35153308,105.00
2,2020-08-14 23:09:10.231,2077862,9257001,34783182,756.00
3,2020-08-18 19:15:48.314,2335664,9259453,34319693,359.99
4,2020-08-17 21:23:46.811,202092,9260633,33685131,180.63


In [6]:
event_df.head()

,created_at,cookie_id,user_id,experiment_group,logged_in,events
0,2020-09-16 17:40:43.692,-9223362196260552664,NaN,0,False,1
1,2020-09-02 10:34:15.637,-9223332603731817103,NaN,0,False,3
2,2020-09-04 05:19:53.226,-9223323544648705373,NaN,0,False,4
3,2020-09-13 17:41:38.381,-9223243877075865587,NaN,0,False,2
4,2020-08-20 07:23:20.641,-9223238861641533575,NaN,0,False,1


# Clean and Transform Data

In [16]:
# use only events from users that we have a user_id for
tracked_event_df = event_df[~event_df['user_id'].isnull()]

# rename created_at columns for easier reference and convert to datetime
tracked_event_df = tracked_event_df.rename(columns={'created_at':'event_fired_at'})
tracked_event_df['event_fired_at'] = pd.to_datetime(tracked_event_df['event_fired_at'])
conversion_df = conversion_df.rename(columns={'created_at':'purchased_at'})
conversion_df['purchased_at'] = pd.to_datetime(conversion_df['purchased_at'])

,event_fired_at,cookie_id,user_id,experiment_group,logged_in,events,purchased_at,order_id,listing_id,revenue,converted
0,2020-08-14 15:31:31.960,-3743295228662505984,5.0,0,True,23,2020-09-03 02:13:25.981,9423992,35591975,65.00,False
1,2020-08-14 03:49:26.389,1801887607634904279,8.0,0,True,591,2020-09-06 17:46:28.971,9451509,35577739,38.00,False
2,2020-08-14 03:49:26.389,1801887607634904279,8.0,0,True,591,2020-09-18 12:39:09.772,9540830,34847672,4.99,False
3,2020-08-14 03:49:26.389,1801887607634904279,8.0,0,True,591,2020-09-21 13:23:49.341,9562822,35869715,200.00,False
4,2020-08-14 03:49:26.389,1801887607634904279,8.0,0,True,591,2020-09-18 17:38:58.277,9543011,35825399,95.00,False


# Merge dataframes

In [27]:
# merge dataframes
df = pd.merge(tracked_event_df, conversion_df, left_on='user_id', right_on='user_id')

# determine conversion
df['converted'] = df['purchased_at'].between(df['event_fired_at'],
                                            df['event_fired_at'] + pd.Timedelta(2, 'd'))
df.head()

,event_fired_at,cookie_id,user_id,experiment_group,logged_in,events,purchased_at,order_id,listing_id,revenue,converted
0,2020-08-14 15:31:31.960,-3743295228662505984,5.0,0,True,23,2020-09-03 02:13:25.981,9423992,35591975,65.00,False
1,2020-08-14 03:49:26.389,1801887607634904279,8.0,0,True,591,2020-09-06 17:46:28.971,9451509,35577739,38.00,False
2,2020-08-14 03:49:26.389,1801887607634904279,8.0,0,True,591,2020-09-18 12:39:09.772,9540830,34847672,4.99,False
3,2020-08-14 03:49:26.389,1801887607634904279,8.0,0,True,591,2020-09-21 13:23:49.341,9562822,35869715,200.00,False
4,2020-08-14 03:49:26.389,1801887607634904279,8.0,0,True,591,2020-09-18 17:38:58.277,9543011,35825399,95.00,False


# Does the data show visitors purchasing more than one item?

In [38]:
user_grouped = df.groupby('user_id')
user_grouped.size().value_counts().sort_index()

1      46249
2      12343
3       5130
4       2692
5       1540
       ...  
121        1
127        1
153        1
168        1
181        1
Length: 64, dtype: int64

absolutely, but...

In [50]:
user_df = user_grouped.get_group(5303450.0)
user_df

,event_fired_at,cookie_id,user_id,experiment_group,logged_in,events,purchased_at,order_id,listing_id,revenue,converted
69026,2020-08-15 17:53:05.602,8120778930691635636,5303450.0,0,False,230,2020-08-25 22:22:11.782,9360379,1433954,51.96,False
69027,2020-08-15 17:53:05.602,8120778930691635636,5303450.0,0,False,230,2020-09-18 21:29:48.543,9544731,35452502,25.99,False
69028,2020-08-15 17:53:05.602,8120778930691635636,5303450.0,0,False,230,2020-08-22 02:43:04.914,9331632,35331187,1000.00,False
69029,2020-08-15 17:53:05.602,8120778930691635636,5303450.0,0,False,230,2020-09-04 01:33:22.816,9432177,35519549,100.00,False
69030,2020-08-15 17:53:05.602,8120778930691635636,5303450.0,0,False,230,2020-08-31 00:21:34.373,9399637,35459950,1250.00,False


In [51]:
user_df['converted'].any()

False

Yes, this means that, for each user that did purchase something within the 2 day window, we'll want to sum their revenue to understand the total, but filter out everything else.

For those who did not make a purchase during that time, we'll want to set revenue = 0 and maintain that they were not "converted"

In [54]:
results = []

# collapse data per the thinking above
for user_id, user_df in tqdm(user_grouped):
    # did user ever purchase something in that time window?
    converted = user_df['converted'].any()
    
    if converted:
        revenue = user_df[user_df['converted']]['revenue'].sum()
    else:
        revenue = 0
        
    result = {'user_id':user_id,
              'total_revenue':revenue,
              'experiment_group':user_df['experiment_group'].iloc[0],
              'converted':converted,
              }
    results.append(result)
    
r_df = pd.DataFrame(results)

  0%|          | 0/71443 [00:00<?, ?it/s]

Note: I'll want to reformat this dataframe when thinking about logged in/out users

An additional note is that it seems that about 141,445 users made purchases during this time and 301,542 were included in this experiment, but only 71,443 users made purchases AND were in the experiment

For much of this notebook, I will be following this kind of decision tree to determine which test I will use which came from a really helpful [blog post on A/B testing](https://towardsdatascience.com/a-b-testing-a-complete-guide-to-statistical-testing-e3f1db140499)

![Statistical Test Decision Tree](https://miro.medium.com/max/700/1*Vzkwzrs4DOmBBa1LymW-PQ.png)

Given that the number of users represented in this dataset is > 1000 (71,443)

# Q1: How did the *conversion rate* change?

In [74]:
conv_df = r_df.groupby('experiment_group').agg({'converted':['mean','std']})
conv_df.columns = ['conversion_rate', 'std_dev']
conv_df

,conversion_rate,std_dev
experiment_group,,
0,0.382096,0.485907
1,0.389880,0.487730


In [75]:
conv_df.loc[1] - conv_df.loc[0]

conversion_rate    0.007784
std_dev            0.001823
dtype: float64

What we see here is that the conversion rates are generally prett close, though the conversion rate for the treatment group, the group that saw the new feature, was **0.77\% higher** than the control group that did not see the new feature.

## Q1A: How confident can we be in this answer?
To answer this question, we'll follow the diagram above and use a **Pearson's Chi-squared test**, because conversions are a discrete value, one either converted or they did not. 

In [78]:
ct_df = pd.crosstab(r_df['experiment_group'], r_df['converted'])
cont_table = ct_df.T
cont_table.head()

experiment_group,0,1
converted,,
False,21846,22018
True,13509,14070


In [83]:
chi2_val, p_val = stats.chi2_contingency(cont_table, correction=False)[:2]


print("- Pearson's chi-squared t-test:")
print(f"   - χ2 value: {chi2_val:.3f}")
print(f"   - p-value: {p_val:.4f}")

- Pearson's chi-squared t-test:
   - χ2 value: 4.566
   - p-value: 0.0326


With a significance level of 0.1, we can say that we'll reject the null hypothesis that there is no difference between those who were in the treatment group and those who were in the control group.

This means that we have evidence to support that the website change will result in a **higher conversion rate**, but does this mean more revenue over all?

# Q2: How did *conversion value* change?

In [88]:
conv_val_df = r_df.groupby('experiment_group').agg({'total_revenue':['mean','std']})
conv_val_df

total_revenue             
                          mean          std
experiment_group                           
0                   159.993679   479.911560
1                   191.350091  4317.025083

In [93]:
total_rev_diff = conv_val_df.loc[1][('total_revenue','mean')] - conv_val_df.loc[0][('total_revenue','mean')]
print(f'The treatment group spent an average of ${total_rev_diff:.2f} more than the control group')

The treatment group spent an average of $31.36 more than the control group


The answer here is yes... but the huge variance among those who were in the treatment group is suspicious. 
We are definitely going to need to assess our confidence in that answer.

For this, I decided to use [*Welch's t-test*](https://en.wikipedia.org/wiki/Welch%27s_t-test) to measure the difference for the following reasons:
* The values are continuous
* We have a very large sample
* We don't know the population variance
* We do know the sample variance, BUT they are very dissimilar between the two groups (479.9 vs. 4317.03)

In [113]:
treatment_group_rev = r_df[r_df['experiment_group']==1]['total_revenue']
control_group_rev =r_df[r_df['experiment_group']==0]['total_revenue']

t_stat, p_val = stats.ttest_ind(treatment_group_rev, control_group_rev, equal_var=False)
print(f't-statistic = {t_stat:0.5f}')
print(f'p-value = {p_val:0.5f}')

t-statistic = 1.37120
p-value = 0.17032


While the t-statistic confirms that the numbers are slightly higher, the p-value challenges this by telling us that we've failed to meet the criteria for rejecting the null hypothesis.

So, ***there is no significant difference*** between the revenue earned by those in the treatment group and those in the control group

# Q3: Are there any differences in behavior between logged in and logged out users?

I'll consider this question, independent of whether they were in the control or treatment group

My first question is, do we have data that shows the same user making purchase both logged in AND logged out?

In [116]:
df.groupby('user_id').agg({'logged_in':np.mean}).value_counts()

logged_in
1.0          39153
0.0          32290
dtype: int64

No, all of the data indicates that users are EITHER logged in or logged out. Great, this makes our analysis easier from now on.

In [118]:
results = []

# collapse data per the thinking above
for user_id, user_df in tqdm(user_grouped):
    # did user ever purchase something in that time window?
    converted = user_df['converted'].any()
    
    if converted:
        revenue = user_df[user_df['converted']]['revenue'].sum()
    else:
        revenue = 0
        
    result = {'user_id':user_id,
              'total_revenue':revenue,
              'experiment_group':user_df['experiment_group'].iloc[0],
              'logged_in':user_df['logged_in'].iloc[0],
              'converted':converted,
              }
    results.append(result)
    
r_df = pd.DataFrame(results)

  0%|          | 0/71443 [00:00<?, ?it/s]

### First, let's look at conversion rate

In [122]:
conv_df = r_df.groupby('logged_in').agg({'converted':['mean','std']})
conv_df.columns = ['conversion_rate', 'std_dev']
conv_df

,conversion_rate,std_dev
logged_in,,
False,0.503995,0.499992
True,0.288739,0.453182


In [124]:
ct_df = pd.crosstab(r_df['logged_in'], r_df['converted'])
ct_df.head()

converted,False,True
logged_in,,
False,16016,16274
True,27848,11305


In [121]:
ct_df = pd.crosstab(r_df['logged_in'], r_df['converted'])
cont_table = ct_df.T

chi2_val, p_val = stats.chi2_contingency(cont_table, correction=False)[:2]


print("- Pearson's chi-squared t-test:")
print(f"   - χ2 value: {chi2_val:.3f}")
print(f"   - p-value: {p_val:.4f}")

- Pearson's chi-squared t-test:
   - χ2 value: 3459.526
   - p-value: 0.0000


Wow, this is very counter-intuitive given that I didn't even know that you could make purchases without logging in. If I am reading this correctly, users who are **not logged in** will purchase within 2 days nearly 50.3\% of the time, while users who are **logged in** only convert 28.8\% of the time.

I'm very curious to hear theories on why this might be the case.

## Let's look at conversion value now

In [132]:
conv_val_df = r_df.groupby('logged_in').agg({'total_revenue':['mean','std']})
conv_val_df

total_revenue             
                   mean          std
logged_in                           
False        198.552926   761.649680
True         157.095105  4111.807454

In [134]:
total_rev_diff = conv_val_df.loc[True][('total_revenue','mean')] - conv_val_df.loc[False][('total_revenue','mean')]
print(f'The logged in group spent an average of ${np.abs(total_rev_diff):.2f} less than the logged out group')

The logged in group spent an average of $41.46 less than the logged out group


In [137]:
treatment_group_rev = r_df[r_df['logged_in']==True]['total_revenue']
control_group_rev =r_df[r_df['logged_in']==False]['total_revenue']

t_stat, p_val = stats.ttest_ind(treatment_group_rev, control_group_rev, equal_var=False)
print(f't-statistic = {t_stat:0.5f}')
print(f'p-value = {p_val:0.5f}')

t-statistic = -1.95481
p-value = 0.05061


Ok, so our logged out users are surely spending more money too. I'd be very curious to learn more about why logged out users spend so much more many and make so many more purchases than logged in users.
* Are these impulse purchases?
* Are there many vintage gear collectors who just don't want to make another account on another website, so they are holding out?

In the future, I would love to take a look at the effect that the new tab feature has on these numbers.

# Final Recommendations

In this analysis, I learned three main things:
1. Implementing a change where clicking on new items will result in a new tab, the conversion rate increases by about 2\%
2. However, there doesn't seem to be a significant change in **conversion value**
3. Users who are logged out seem to spend way more money

In general, I would not recommend making this change. There is no evidence to support that it will result in higher revenue and the change in conversion rate is only 2\%. That said, there may be reasons to care about conversion rate even when conversion value change is insignificant. Perhaps, we believe that just getting people out there and making purchases will increase the word of mouth traffic that gets more users to the site down the road. Personally, if this were the primary goal, I would want to implement a change that does better than 2\% increase, but perhaps its one change among a battery of other changes that could achieve that.

Thank you for this opportunity to play with the Reverb data. I had a lot of fun and learned a lot in the process!

Cheers,
Scott